In [2]:
cd /content/drive/MyDrive/20210611任务

/content/drive/MyDrive/20210611任务


In [ ]:
!pip install transformers sentencepiece datasets rouge_score

初步设想建立语义关系三元组（Q S A）
Q：问题
S：含义
A：答案文本

A方案：
1.   利用Bert对医学样本建立词向量库
2.   使用Transformer对A进行S提取
3.   对比Q与S库的欧式距离，从而选出答案


B方案：
1.   利用Bert对医学样本建立词向量库
2.   使用GPT2增强Q样本
3.   Transformer对A进行S1提取
4.   Transformer对Q进行S2提取
5.   对比S1与S2的欧式距离，从而选出答案

In [4]:
import numpy as np
import pandas as pd
from pathlib import Path 

import os

import torch
import torch.optim as optim

import random 
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from transformers import AdamW
from functools import partial

In [5]:
from sklearn.model_selection import train_test_split
f = open('./diseaseBERT/diseaseKnowledgeInfusionTraining/data/extractedQuestionAnswers_total_removeNoisy_maskedLM.txt','r')
total_text = f.read().split('\n')[:-1]

total_train,total_test,_,_ = train_test_split(total_text,[0 for line in total_text],test_size=0.2,random_state=66)

In [6]:
S_train_dic = {}
Q_train_dic = {}
A_train_dic = {}
for key, line in enumerate(total_train):
  SQA_temp = line.split('?\t')
  SQ_temp = SQA_temp[0].split('\t')
  S_train_dic[key] = SQ_temp[0] + ' : ' + SQ_temp[1]
  Q_train_dic[key] = SQ_temp[2]
  A_train_dic[key] = SQA_temp[1]

S_test_dic = {}
Q_test_dic = {}
A_test_dic = {}
for key, line in enumerate(total_test):
  SQA_temp = line.split('?\t')
  SQ_temp = SQA_temp[0].split('\t')
  S_test_dic[key] = SQ_temp[0] + ' : ' + SQ_temp[1]
  Q_test_dic[key] = SQ_temp[2]
  A_test_dic[key] = SQA_temp[1]

In [7]:
import pandas as pd

data = pd.DataFrame({'S':list(S_train_dic.values()),'Q':list(Q_train_dic.values()),'A':list(A_train_dic.values())})
data

,S,Q,A
0,gangrene : treatments,what are the treatments of gangrene,treatment varies based on the severity and typ...
1,multiple endocrine neoplasia type 1 : treatments,what are the treatments of multiple endocrine ...,the treatment of choice of parathyroid tumors ...
2,pancreatic pseudocyst : diagnosis,what are the diagnosis of pancreatic pseudocyst,diagnosis of pancreatic pseudocyst can be base...
3,oral submucous fibrosis : physiology,what are the physiology of oral submucous fibr...,"""exposure to areca nut (arecacatechu) containi..."
4,thrombophilia : diagnosis,what are the diagnosis of thrombophilia,tests for thrombophilia include complete blood...
...,...,...,...
11688,behçet's disease : treatments,what are the treatments of behçet's disease,current treatment is aimed at easing the sympt...
11689,tetany : general,what is tetany,tetany or tetanic seizure is a medical sign co...
11690,avulsion fracture : treatments,what are the treatments of avulsion fracture,"if the fracture is small, it is usually suffic..."
11691,lichen planus : treatments,what are the treatments of lichen planus,"there is no cure for lichen planus, when medic..."


# GPT2

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextGenerationPipeline

model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [8]:
QStrain_data = []
QStest_data = []

for indexs in range(len(list(Q_train_dic.values()))):
  QStrain_data.append([Q_train_dic[indexs],S_train_dic[indexs]])
for indexs in range(len(list(Q_test_dic.values()))):
  QStest_data.append([Q_test_dic[indexs],S_test_dic[indexs]])

AStrain_data = []
AStest_data = []

for indexs in range(len(list(A_train_dic.values()))):
  AStrain_data.append([A_train_dic[indexs],S_train_dic[indexs]])
for indexs in range(len(list(A_test_dic.values()))):
  AStest_data.append([A_test_dic[indexs],S_test_dic[indexs]])

In [64]:
from tqdm.auto import tqdm
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(4)
all_data = []
for line in tqdm(QStrain_data):
  qTemp = line[0].split(" ")
  qTemp = generator(" ".join(qTemp[:len(qTemp)//3 * 2]), max_length=16, num_return_sequences=10)
  for dic in qTemp:
    all_data.append([dic['generated_text'].replace("\n",""), line[1]])
QStrain_data = all_data

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,Seq2SeqTrainingArguments
# model_name = "google/roberta2roberta_L-24_cnn_daily_mail"
model_name = "t5-small"
# model_name = '/content/drive/MyDrive/20210611任务/AS-summarization/checkpoint-2000'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Qtrain_encoding = tokenizer(
    list(Q_train_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Strain_label_encoding = tokenizer(
    list(S_train_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Qtest_encoding = tokenizer(
    list(Q_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)

Stest_label_encoding = tokenizer(
    list(S_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=16,
    return_tensors="pt"
)


Atrain_encoding = tokenizer(
    list(A_train_dic.values()),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

Atest_encoding = tokenizer(
    list(A_test_dic.values()),
    padding=True,
    truncation=True,
    max_length=256,
    return_tensors="pt"
)

# A_batch = tokenizer(
#     list(A_dic.values()),
#     padding=True,
#     truncation=True,
#     max_length=512,
#     return_tensors="pt"
# )

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [ ]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
 
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
 
    def __len__(self):
        return len(self.labels)
 
QStrain_dataset = IMDbDataset(Qtrain_encoding, Strain_label_encoding['input_ids'])
QStest_dataset = IMDbDataset(Qtest_encoding, Stest_label_encoding['input_ids'])

AStrain_dataset = IMDbDataset(Atrain_encoding, Strain_label_encoding['input_ids'])
AStest_dataset = IMDbDataset(Atest_encoding, Stest_label_encoding['input_ids'])

In [ ]:
import nltk
import numpy as np
from datasets import load_metric
nltk.download('punkt')
metric = load_metric("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
#QS
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    "QS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=QStrain_dataset,
    eval_dataset=QStest_dataset,
    # data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Using amp fp16 backend
***** Running training *****
  Num examples = 11693
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1098
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1349: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default b

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.089500,0.070495,97.893800,95.936800,97.868600,97.859600,9.826900
2,0.070200,0.040085,98.178400,96.655700,98.185900,98.180800,9.861100
3,0.027600,0.036042,98.212200,96.704500,98.217000,98.210900,9.868300


***** Running Evaluation *****
  Num examples = 2924
  Batch size = 32
Saving model checkpoint to QS-summarization/checkpoint-500
Configuration saved in QS-summarization/checkpoint-500/config.json
Model weights saved in QS-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in QS-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in QS-summarization/checkpoint-500/special_tokens_map.json
Copy vocab file to QS-summarization/checkpoint-500/spiece.model
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rat

TrainOutput(global_step=1098, training_loss=0.8429933545096542, metrics={'train_runtime': 158.5054, 'train_samples_per_second': 221.311, 'train_steps_per_second': 6.927, 'total_flos': 148364220432384.0, 'train_loss': 0.8429933545096542, 'epoch': 3.0})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    "QS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=QStrain_dataset,
    eval_dataset=QStest_dataset,
    # data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 11693
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1098
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.025300,0.028251,98.450500,97.241300,98.447300,98.446900,9.885400
2,0.035800,0.022288,98.631100,97.530700,98.635200,98.623700,9.888900
3,0.013600,0.021125,98.678600,97.660300,98.681100,98.668200,9.893000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1349: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 2924
  Batch size = 32
Saving model checkpoint to QS-summarization/checkpoint-500
Configuration saved in QS-summarization/checkpoint-500/config.json
Model weights saved in QS-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in QS-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in QS-summarization/checkpoint-500/special_tokens_map.json
Copy vocab file to QS-summarization/checkpoint-500/spiece.model
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct f

TrainOutput(global_step=1098, training_loss=0.0329151949310889, metrics={'train_runtime': 149.6737, 'train_samples_per_second': 234.37, 'train_steps_per_second': 7.336, 'total_flos': 148364220432384.0, 'train_loss': 0.0329151949310889, 'epoch': 3.0})

In [ ]:
training_args = Seq2SeqTrainingArguments(
    "QS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=QStrain_dataset,
    eval_dataset=QStest_dataset,
    # data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 11693
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2193
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().

RuntimeError: ignored

In [ ]:
features_in_hook = []
features_out_hook = []

def hook(module, fea_in, fea_out):
    features_in_hook.append(fea_in)
    features_out_hook.append(fea_out)
    return None

layer_name = 'decoder'
for (name, module) in model.named_modules():
    if name == layer_name:
        module.register_forward_hook(hook=hook)

In [ ]:
#AS
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    "AS-summarization",
    evaluation_strategy = "epoch",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=AStrain_dataset,
    eval_dataset=AStest_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 11693
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 35079
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().d

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.650500,0.493238,74.885200,62.177600,74.792600,74.872800,9.326600


Saving model checkpoint to AS-summarization/checkpoint-500
Configuration saved in AS-summarization/checkpoint-500/config.json
Model weights saved in AS-summarization/checkpoint-500/pytorch_model.bin
tokenizer config file saved in AS-summarization/checkpoint-500/tokenizer_config.json
Special tokens file saved in AS-summarization/checkpoint-500/special_tokens_map.json
Copy vocab file to AS-summarization/checkpoint-500/spiece.model
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
Saving model checkpoint to AS-s

KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline
# model.device('cpu')
processor = pipeline('summarization', tokenizer=tokenizer, model=model.cpu())

In [ ]:
result = []

for i in tqdm(range(len(Qtest_encoding["input_ids"])))
  model.generate(Qtest_encoding["input_ids"][i:i+1].cuda(), do_sample=True, num_return_sequences=1, output_scores=True)
  result.append(features_out_hook[0]['last_hidden_state'])
  features_out_hook = []

In [ ]:
pd.DataFrame({"Prediction":result}).to_csv("Q2S.csv")